<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Getting started tutorial

In [1]:
from pyspark.sql import SparkSession

def calculate_red_violations(data_source, output_uri):
    """
    Processes sample food establishment inspection data and queries the data to find the top 10 establishments
    with the most Red violations from 2006 to 2020.

    :param data_source: The URI of your food establishment data CSV, such as 's3://DOC-EXAMPLE-BUCKET/food-establishment-data.csv'.
    :param output_uri: The URI where output is written, such as 's3://DOC-EXAMPLE-BUCKET/restaurant_violation_results'.
    """
    with SparkSession.builder.appName("Calculate Red Health Violations").getOrCreate() as spark:
        # Load the restaurant violation CSV data
        if data_source is not None:
            restaurants_df = spark.read.option("header", "true").csv(data_source)

        # Create an in-memory DataFrame to query
        restaurants_df.createOrReplaceTempView("restaurant_violations")

        # Create a DataFrame of the top 10 restaurants with the most Red violations
        top_red_violation_restaurants = spark.sql("""SELECT name, count(*) AS total_red_violations 
          FROM restaurant_violations 
          WHERE violation_type = 'RED' 
          GROUP BY name 
          ORDER BY total_red_violations DESC LIMIT 10""")

        # Write the results to the specified output URI
        top_red_violation_restaurants.write.option("header", "true").mode("overwrite").csv(output_uri)

# Set the values for data_source and output_uri
data_source = 's3://gelcawsemr/food_establishment_data.csv'
output_uri = 's3://gelcawsemr/restaurant_violation_results'

# Call the function with the provided values
calculate_red_violations(data_source, output_uri)


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1701743246151_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Programme to uncompress the archives

In [ ]:
import boto3
import tarfile
import bz2
import os

# Set up AWS credentials
aws_access_key_id = 'YOUR_ACCESS_KEY_ID'
aws_secret_access_key = 'YOUR_SECRET_ACCESS_KEY'
region_name = 'YOUR_REGION_NAME'

# Set up S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

# Set up EMR client
emr = boto3.client('emr', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

# Set up the job flow
job_flow_id = 'YOUR_JOB_FLOW_ID'

# Set up the source and destination S3 bucket names
source_bucket_name = 'YOUR_SOURCE_BUCKET_NAME'
destination_bucket_name = 'YOUR_DESTINATION_BUCKET_NAME'

# Set up the S3 bucket and file paths
tar_file_key = 'path/to/your/tar/file.tar'

# Fetch the .tar file from the source S3 bucket
s3.download_file(source_bucket_name, tar_file_key, 'file.tar')

# Uncompress the .tar file
with tarfile.open('file.tar', 'r') as tar:
    tar.extractall()

# Iterate over the extracted files
for root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.bz2'):
            # Uncompress each .bz2 file
            with bz2.open(os.path.join(root, file), 'rb') as bz_file:
                uncompressed_data = bz_file.read()
                # Process the uncompressed data (e.g., write to a new file, parse JSON, etc.)
                # Your processing logic goes here
                
                # Get the relative path of the file within the directory tree
                relative_path = os.path.relpath(os.path.join(root, file), '.')

                # Upload the processed file to the destination S3 bucket with the same directory tree structure
                destination_key = os.path.join(relative_path, file)
                s3.put_object(Body=uncompressed_data, Bucket=destination_bucket_name, Key=destination_key)

# Terminate the EMR cluster
emr.terminate_job_flows(JobFlowIds=[job_flow_id])

## Sample code to parse the tweets files ([JSONL](https://spark.apache.org/docs/latest/sql-data-sources-json.html) format) to a dataframe over Amazon EMR

In [ ]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("Tweet Analysis") \
    .getOrCreate()

# Read the JSONL files from S3 and create a dataframe
tweets_df = spark.read.json("s3://your-bucket/tweets.jsonl", multiLine=True)

# Filter the dataframe to extract the desired data
filtered_df = tweets_df.filter(tweets_df['text'].contains("your-filter-keyword"))

# Show the filtered dataframe
filtered_df.show()

# snscrape format

In [72]:
import pandas as pd

# Read the JSON file into a DataFrame
df1 = pd.read_json('snscrape.json', lines=True)

In [74]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   _type             200 non-null    object             
 1   url               200 non-null    object             
 2   date              200 non-null    datetime64[ns, UTC]
 3   rawContent        200 non-null    object             
 4   renderedContent   200 non-null    object             
 5   id                200 non-null    int64              
 6   user              200 non-null    object             
 7   replyCount        200 non-null    int64              
 8   retweetCount      200 non-null    int64              
 9   likeCount         200 non-null    int64              
 10  quoteCount        200 non-null    int64              
 11  conversationId    200 non-null    int64              
 12  lang              200 non-null    object             
 13  sourc

In [76]:
import json

quoted_tweet = df1.loc[7, 'quotedTweet']
quoted_tweet_str = json.dumps(quoted_tweet)
quoted_tweet_dict = json.loads(quoted_tweet_str)

media = quoted_tweet_dict['media']
print(media)

[{'_type': 'snscrape.modules.twitter.Photo', 'previewUrl': 'https://pbs.twimg.com/media/DyLY0iQW0AAk5XK?format=jpg&name=small', 'fullUrl': 'https://pbs.twimg.com/media/DyLY0iQW0AAk5XK?format=jpg&name=large'}]


In [78]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   _type             200 non-null    object             
 1   url               200 non-null    object             
 2   date              200 non-null    datetime64[ns, UTC]
 3   rawContent        200 non-null    object             
 4   renderedContent   200 non-null    object             
 5   id                200 non-null    int64              
 6   user              200 non-null    object             
 7   replyCount        200 non-null    int64              
 8   retweetCount      200 non-null    int64              
 9   likeCount         200 non-null    int64              
 10  quoteCount        200 non-null    int64              
 11  conversationId    200 non-null    int64              
 12  lang              200 non-null    object             
 13  sourc

# Internet Archive format

In [80]:
import pandas as pd

# Read the JSON file into a DataFrame
df2 = pd.read_json('intarch.json', lines=True)

/tmp/ipykernel_115/2443123195.py:4: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df2 = pd.read_json('intarch.json', lines=True)
/tmp/ipykernel_115/2443123195.py:4: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df2 = pd.read_json('intarch.json', lines=True)


In [82]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   created_at                 176 non-null    datetime64[ns, UTC]
 1   id                         176 non-null    float64            
 2   id_str                     176 non-null    float64            
 3   text                       176 non-null    object             
 4   display_text_range         45 non-null     object             
 5   source                     176 non-null    object             
 6   truncated                  176 non-null    float64            
 7   in_reply_to_status_id      38 non-null     float64            
 8   in_reply_to_status_id_str  38 non-null     float64            
 9   in_reply_to_user_id        39 non-null     float64            
 10  in_reply_to_user_id_str    39 non-null     float64            
 11  in_rep

In [84]:
df2

,created_at,id,id_str,text,display_text_range,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,timestamp_ms,retweeted_status,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_tweet,delete
0,2019-01-01 07:29:00+00:00,1.080003e+18,1.080003e+18,@supr_dorapo 最新のゼルダやってないけどゼルダも好きですﾋﾟｭｰﾋﾟｭｰﾋﾟｭﾋ...,"[13, 58]","<a href=""http://twitter.com/download/android"" ...",0.0,1.080003e+18,1.080003e+18,2.787406e+09,...,2019-01-01 07:28:07.168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01 07:29:00+00:00,1.080003e+18,1.080003e+18,そうだよ、僕がピーマンだよ,NaN,"<a href=""http://twittbot.net/"" rel=""nofollow"">...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01 07:29:00+00:00,1.080003e+18,1.080003e+18,@subaru_yamamoto 勝ち退くが利って、それ得意なひとTwitterにたくさんい...,"[17, 66]","<a href=""http://twitter.com/download/iphone"" r...",0.0,1.080002e+18,1.080002e+18,3.955496e+09,...,2019-01-01 07:28:07.168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01 07:29:00+00:00,1.080003e+18,1.080003e+18,RT @PPGcnjp: 初もうで😊 https://t.co/NOfd4q1ro5,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,{'created_at': 'Tue Jan 01 07:24:00 +0000 2019...,"{'media': [{'id': 1076011587624873984, 'id_str...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01 07:29:00+00:00,1.080003e+18,1.080003e+18,RT @DJ_GINTA_NEW: 【お正月の歌作ったよん🎍】 https://t.co/e...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,{'created_at': 'Tue Jan 01 07:22:06 +0000 2019...,"{'media': [{'id': 1080001090727247872, 'id_str...",0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2019-01-01 07:29:04+00:00,1.080003e+18,1.080003e+18,@Fams000 @Rouguii77 +mention inch'allah bonne ...,"[20, 59]","<a href=""http://twitter.com/download/android"" ...",0.0,1.079895e+18,1.079895e+18,2.878622e+09,...,2019-01-01 07:28:07.168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,2019-01-01 07:29:04+00:00,1.080003e+18,1.080003e+18,RT @Sphenodontiaart: hope to god they kiss for...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,{'created_at': 'Mon Dec 31 06:09:07 +0000 2018...,"{'media': [{'id': 1079620371505065984, 'id_str...",0.0,NaN,NaN,NaN,NaN,NaN,NaN
197,2019-01-01 07:29:04+00:00,1.080003e+18,1.080003e+18,RT @fuzakefactory: 明けましておめでとうございます🐗\n新年お年玉企画🎉\...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,{'created_at': 'Tue Jan 01 06:45:32 +0000 2019...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,2019-01-01 07:29:04+00:00,1.080003e+18,1.080003e+18,Feliz año nuevo! Happy new year! #2019 @ Nuevo...,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
